# PART 1 : Dataset Analysis with VSC

## 1.1. Import des librairies

In [ ]:
# Loading librairies needed
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statistics

: 

## 1.2 Exploration de la Dataframe

In [ ]:
# Loading the dataset 
dataset = pd.read_excel("src/get_around_delay_analysis.xlsx")

: 

In [ ]:
#dataset.shape
dataset.shape

: 

In [ ]:
# Visualazing the first 20 rows of the dataset
dataset.head(20)

: 

In [ ]:
# Looking for more analysis of the dataset due to describe function
dataset.describe(include='all')

: 

In [ ]:
# NOTE_ : Bellow we can see that some feature have missing value, let's check the number of it

: 

1.2.1. Prépararion au data cleaning 
* 'rental_id' : from this column, we just can count the number of rental. It is an ID column, we have to delete it from the dataset
* 'car_id' : from this columnm, we just can count the number unique of car of the dataset
* 'checkin_type' : from this column, we have 3 choices : 'mobile', 'connect' and 'paper'. This is different way to manage rental car. We can see the repartition of this columns to find a correlation between late rental and checkin type
* 'state' : here, we have the state of the rental car. We have 2 choices : 'ended', 'cancelled'. We will analyse this column
* 'delay_at_checkout_in_minutes' : this column is not enought clear. we have to tranforme data in this column to understand better the information that this coluns bring to us.
* 'previous_ended_rental_id' : this column is filled only when the rental car ID have a rental after. So that why is it not filled and have missing value. We have to keep this information to further analysis
* 'time_delta_with_previous_rental_in_minutes' : this column is not clear for a client. This column count the delta time between the time of rental and the time this rental have to be return. This is an interesting column to analyze and determine a threshold.

#### 1.2.2. Gestion des valeurs manquantes

In [ ]:
# Let's see if we have missing value in the dataset
dataset.isnull().sum()

: 

In [ ]:
# NOTE_ :
# We can see that only 'delay_at_checkout_in_minutes',
# 'previous_ended_rental_id','time_delta_with_previous_rental_in_minutes' have some missing value.
# But delay_at_checkout_in_minutes is normal to have missing value because missing value mean that 
# at the rental there is no delay, so we have to keep this value and change it.
#'previous_ended_rental_id' and 'time_delta_with_previous_rental_in_minutes' are correlated. 
# Under 21310 rental ID there is 19469 were that don't have previous rental car. 
# So we can keep missing value and change it into zero.

: 

In [ ]:
# Filling the Nan value on 'delay_at_checkout_in_minutes' missing. We can still keep this for unknow 
# value and keep more informations
dataset.delay_at_checkout_in_minutes = dataset.delay_at_checkout_in_minutes.fillna(0)
dataset.previous_ended_rental_id = dataset.previous_ended_rental_id.fillna(0)
dataset.time_delta_with_previous_rental_in_minutes = dataset.time_delta_with_previous_rental_in_minutes.fillna(0)

: 

In [ ]:
# Checking if the tranformation has been well done
dataset.head(20)

: 

In [ ]:
# Great, now as we finish to manage missing value, we can modify some value to analyse more simply the information

: 

#### 1.2.3. Applications des transformations

In [ ]:
# Creating a new column for classification previous_rental yes or no
dataset['previous_rental'] = dataset.previous_ended_rental_id.apply(lambda x : "No" if x==0 else "Yes")

: 

In [ ]:
# Creating a new column for classification delay or not delay
dataset['is_delay'] = dataset.delay_at_checkout_in_minutes.apply(lambda x : "No" if x>=0 else "Yes")

: 

In [ ]:
# Checking transformations
dataset.head(50)

: 

In [ ]:
# Checking type value to manipulate correctly the dataset
dataset.dtypes

: 

In [ ]:
# From float to object types for rental_id, car_id and previous_ended_rental_id
dataset.rental_id = dataset.rental_id.astype('object')
dataset.car_id = dataset.car_id.astype('object')
dataset.previous_ended_rental_id = dataset.previous_ended_rental_id.astype('object')

: 

In [ ]:
dataset.dtypes

: 

In [ ]:
# NOTE_ : There is some outlayers in the dataset. Let's create time category

: 

In [ ]:
dataset['delay_types'] = dataset['delay_at_checkout_in_minutes'].apply(lambda x : 'No_delay' if x == 0 else
                                                                                ('Less than an hours' if x >0 and x<60 else
                                                                                ('Less than an hours' if x <0 and x>-60 else
                                                                                ('1h to 3h' if x >=60 and x<180 else
                                                                                ('1h to 3h' if x <=-60 and x>-180 else
                                                                                ('3h to 6h' if x>=180 and x<360 else
                                                                                ('3h to 6h' if x<=-180 and x>-360 else
                                                                                ("6h to 12h" if x>=360 and x<720 else
                                                                                ('6h to 12h' if x<=-360 and x>-720 else
                                                                                ("12h to 24h"if x>=720 and x<1440 else
                                                                                ("12h to 24h"if x<=-720 and x>-1440 else
                                                                                ("One day" if x==1440 else
                                                                                ("One day" if x==-1440 else
                                                                                ("Two day" if x>1440 and x<2880 else
                                                                                ("Two day" if x<-1440 and x>-2880 else 
                                                                                "More than 3 days"
                                                                                )))))))))))))))

: 

In [ ]:
dataset['time_delta'] = dataset['time_delta_with_previous_rental_in_minutes'].apply(lambda x : 'No_time_delta' if x == 0 else
                                                                                ('Less than an hours' if x >0 and x<60 else
                                                                                ('Less than an hours' if x <0 and x>-60 else
                                                                                ('1h to 3h' if x >=60 and x<180 else
                                                                                ('1h to 3h' if x <=-60 and x>-180 else
                                                                                ('3h to 6h' if x>=180 and x<360 else
                                                                                ('3h to 6h' if x<=-180 and x>-360 else
                                                                                ("6h to 12h" if x>=360 and x<720 else
                                                                                ('6h to 12h' if x<=-360 and x>-720 else
                                                                                ("12h to 24h"if x>=720 and x<1440 else
                                                                                ("12h to 24h"if x<=-720 and x>-1440 else
                                                                                ("One day" if x==1440 else
                                                                                ("One day" if x==-1440 else
                                                                                ("Two day" if x>1440 and x<2880 else
                                                                                ("Two day" if x<-1440 and x>-2880 else 
                                                                                "More than 3 days"
                                                                                )))))))))))))))

: 

In [ ]:
# Checking the transformation applied
dataset.head()

: 

In [ ]:
# Counting the number of car 
number_of_car = len(set(dataset['car_id']))
number_of_car

: 

In [ ]:
dataset["previous_rental"].value_counts()

: 

In [ ]:
mask = dataset['previous_rental'] == 'Yes'

df = dataset.loc[mask,:]

: 

In [ ]:
df.head()

: 

In [ ]:
len(df['car_id'].unique().tolist())

: 

In [ ]:
df["car_id"].value_counts()

: 

In [ ]:
df["car_id"].value_counts().mean()

: 

In [ ]:
list_car_ID = df["car_id"].value_counts().to_list()

: 

In [ ]:
from collections import Counter

elem = Counter(list_car_ID)

: 

In [ ]:
print(elem)         ## NOTE_ : FAIRE UN GRAPHIQUE STATISTIQUES DESSUS POIUR VOIR LA REPARTITION 

: 

In [ ]:
print(statistics.median(list_car_ID))

: 

In [ ]:
# Deleting useless columns

: 

In [ ]:
dataset = dataset.drop(columns= ['delay_at_checkout_in_minutes',
                                'previous_ended_rental_id',
                                'time_delta_with_previous_rental_in_minutes'])

: 

In [ ]:
dataset.head(50)

: 

## 1.3. Préparation de Dataframe pour l'analyse

In [ ]:
## Création de Dataframe pour l'analyse

: 

1.1.2.3. Dataframe 1

In [ ]:
# Filter : 
# 'previous_rental' == 'Yes'

mask1 = dataset['previous_rental'] == 'Yes'
data_F1 = dataset.loc[mask1,:]

: 

In [ ]:
data_F1.head()

: 

1.1.2.3. Dataframe 2

In [ ]:
# Filters :
# 'previous_rental = 'yes'
# 'is_delay' = 'Yes'
mask2 = data_F1['is_delay'] == "Yes"
data_F2 = data_F1.loc[mask2,:]

: 

In [ ]:
data_F2.head()

: 

1.1.2.3. Dataframe 3

In [ ]:
# Filters : 
# 'previous_rental' == 'Yes'
# 'state' == 'ended'
mask3 = data_F1["state"] == 'ended'
data_F3 = data_F1.loc[mask3,:]

: 

1.1.2.3. Dataframe 4

In [ ]:
# Filters :
# 'previous_rental' == 'Yes'
# 'is_delay' == 'Yes'
# 'state' == 'ended'

mask4 = data_F2["state"] == 'ended'
data_F4 = data_F2.loc[mask4,:]

: 

1.1.2.3. Dataframe 5

In [ ]:
# Filters : 
# 'state' == 'canceled'

mask5 = dataset['state'] == 'canceled'
data_F5 = dataset.loc[mask5,:]

: 

1.1.2.3. Dataframe 6

In [ ]:
# Filters :
# 'previous_rental' == 'No'
mask6 = dataset['previous_rental'] == 'No'
data_F1 = dataset.loc[mask1,:]

: 

# 1.4. Analyse des Dataframes

## 1.4.1. Question 1 : How often are drivers late for the next check-in ? 

### 1.4.1.1 Analysis

#### 1.4.1.1.1. Graphique 1

In [ ]:
# Graph 1 - To know the distribution of late rental we gonna use de data_F1 (dataset with only previous rental) and x = "is-delay"

fig, ax = plt.subplots(figsize=(10,5))

sns.countplot(data=data_F1, x="is_delay")                               
ax.set_xticklabels(ax.get_xticklabels(), rotation=70)
ax.set_title("Repartition of delay on all previous rental")
total = float(len(data_F1))
for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='center')

: 

In [ ]:
# Quick data analysis
# * As we can see, late rental represent 38% of previous rental against 62% who are not late. This is more than 1/3

: 

### 1.4.1.2 Answer to the question

In [ ]:
### ANSWER TO THE QUESTION ###
# More than 1/3 are late for the next check-in.

: 

## 1.4.2 Question 2 : How does it impact the next driver ?

In [ ]:
## To know how does it impact the next driver we gonna analyze late return

: 

### 1.4.2.1. Analysis

##### 1.4.2.1.1. Graphique 2 : Repartition of state of rental (ended or canceled) on all delay of previous rental

In [ ]:
# Graph 2 - To know how late impact the next driver, we gonna use de data_F2 (dataset with only previous rental and delay) 
# and x = "state"

fig, ax = plt.subplots(figsize=(10,5))

sns.countplot(data=data_F2, x="state")                               
ax.set_xticklabels(ax.get_xticklabels(), rotation=70)
ax.set_title("Repartition of state of rental (ended or canceled) on all delay of previous rental")
total = float(len(data_F2))
for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='center')

: 

In [ ]:
# Graph 2 - Quick data analysis :
# * We can see that all of driver keep their rental even if they are late
# * We can conclude that all next driver are unsatisfied.

: 

##### 1.4.2.1.2 Graphique 3 : Delay_types repartition on delay of previous rental

In [ ]:
# Graph 3 - Now, we gonna see from late rental, how long are they late
# We gonna used data_F2 and x = 'delay_types'

fig, ax = plt.subplots(figsize=(10,5))

sns.countplot(data=data_F2, x="delay_types", order = ["Less than an hours",
                                                        "1h to 3h", "3h to 6h", 
                                                        "6h to 12h","12h to 24h","Two day", 
                                                        "More than 3 days"])                                                    
ax.set_xticklabels(ax.get_xticklabels(), rotation=70)
ax.set_title("Delay_types repartition on delay of previous rental")
total = float(len(data_F2))
for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='center')

: 

In [ ]:
# Graph 3 - Quick data analysis :
# * We can see that most of delay are under 3h late
# * 53% 'Less than an hours'
# * 33 % "1h to 3h"

: 

#### 1.4.2.1.3. Graphique 4 : Time_delta repartition on delay of previous rental

In [ ]:
# Graph 4 - Now, we gonna see from late rental, how was the delay between the original time to give back 
# the rental and the next rental
# We gonna used data_F2 and x = 'time_delta'

fig, ax = plt.subplots(figsize=(10,5))

sns.countplot(data=data_F2, x="time_delta", order = ["No_time_delta","Less than an hours",
                                                    "1h to 3h", "3h to 6h", "6h to 12h",
                                                    "12h to 24h","Two day", "More than 3 days"])                                                    
ax.set_xticklabels(ax.get_xticklabels(), rotation=70)
ax.set_title("Time_delta repartition on delay of previous rental")
total = float(len(data_F2))
for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='center')

: 

In [ ]:
# Graph 4 - Quick data analysis :
# * We can see that mostly when the delta is from 6h to 12h, the car rental are late at 30%
# * Then, it is the time delta of 1h to 3h with 26%
# * Both, represent 56% of late return.
# * When the time delta of 'Less than an hours', we have only 7% of late return.
# * and 13% are late when they don't have time delta
# * We can conclude that if the time delta is short, there are less late return.

: 

#### 1.4.2.1.4. Graphique 5 : Delay_types repartition on the state 'ended' of all previous late rental

In [ ]:
# Graph 5 - Now, we gonna see from late rental, for each state what happend
# We gonna used data_F3 and x = 'delay_types'

fig, ax = plt.subplots(figsize=(10,5))

sns.countplot(data=data_F3, x="delay_types", order = ["No_delay","Less than an hours",
                                                        "1h to 3h", "3h to 6h", "6h to 12h",
                                                        "12h to 24h","Two day", "More than 3 days"])                                                    
ax.set_xticklabels(ax.get_xticklabels(), rotation=70)
ax.set_title("Delay_types repartition on the state 'ended' of all previous late rental")
total = float(len(data_F3))
for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='center')

: 

In [ ]:
# Graph 5 - Quick data analysis :
# * Mostly people are late bellow 3h. It represent 80% of late !

: 

#### 1.4.2.1.5. Graphique 6 : Time_delta repartition on the state 'canceled

In [ ]:
# Graph 6 - Now, we gonna see from late rental, for each state what happend
# We gonna used data_F3 and x = 'time_delta'

fig, ax = plt.subplots(figsize=(10,5))

sns.countplot(data=data_F3, x="time_delta", order = ["No_time_delta","Less than an hours",
                                                    "1h to 3h", "3h to 6h", "6h to 12h","12h to 24h"
                                                    ,"Two day", "More than 3 days"])                                                    
ax.set_xticklabels(ax.get_xticklabels(), rotation=70);
ax.set_title("Time_delta repartition on the state 'canceled'");
total = float(len(data_F3))
for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='center')

: 

In [ ]:
# Graph 6 - Quick data analysis :
# Here we can analyze that mostly of the 'canceled' state, had a gap of 6h to 12h at 35% and 17% with a gap from 3h to 6h. 
# We can notice that this is not only because le previous rental are late. But they just canceled for unknown reason.
# But from no time delta to 3h late represent 42% of cancelation

# To avoid late cancelation, we should put a threshold on it. 
# The threshold of cancelation should be 24h before the time departure

: 

#### 1.4.2.1.6. Graphique 7 : Checkin_type repartition on the state 'canceled'

In [ ]:
# Graph 7 - Now, we gonna see from late rental, if check-in type have an impact on late rental
# We gonna used data_F3 and x = 'checkin_type'

fig, ax = plt.subplots(figsize=(10,5))

sns.countplot(data=data_F3, x="checkin_type")                                                    
ax.set_xticklabels(ax.get_xticklabels(), rotation=70);
ax.set_title("Checkin_type repartition on the state 'canceled'");
total = float(len(data_F3))
for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='center')

: 

In [ ]:
# Graph 7 - Quick data analysis :
# Just to check the checkin_type distribution and as they are almost 50/50, this is not a feature that affect delay.

: 

 #### 1.4.2.1.7. Graphique 8 : Delay_types repartition on the state 'ended'

In [ ]:
# Graph 8 - Now, we gonna see from late rental, if check-in type have an impact on late rental
# We gonna used data_F4 and x = 'delay_types'

fig, ax = plt.subplots(figsize=(10,5))

sns.countplot(data=data_F4, x="delay_types", order = ['No_delay', 'Less than an hours', '1h to 3h', '3h to 6h','6h to 12h', '12h to 24h', 'Two day'])                                                    
ax.set_xticklabels(ax.get_xticklabels(), rotation=70);
ax.set_title("Delay_types repartition on the state 'ended'")
total = float(len(data_F4))
for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='center')

: 

In [ ]:
# Graph 8 - Quick data analysis :
# Most of people who keep their renting, can still rent the car until 3h late. it represent 87% under all delay_types

: 

#### 1.4.2.1.8. Graphique 9 : Time_delta repartition on the state 'ended'

In [ ]:
# Graph 9 - Now, we gonna see from late rental, if check-in type have an impact on late rental
# We gonna used data_F4 and x = 'time_delta'

fig, ax = plt.subplots(figsize=(10,5))

sns.countplot(data=data_F4, x="time_delta", order = ['No_time_delta','Less than an hours','1h to 3h', '3h to 6h', '6h to 12h', '12h to 24h', 'two days'])                                                    
ax.set_xticklabels(ax.get_xticklabels(), rotation=70);
ax.set_title("Time_delta repartition on the state 'ended'");
total = float(len(data_F4))
for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='center')

: 

In [ ]:
# Graph 9 - Quick data analysis :
# Initially, people keep their renting because the time_delta is longer than other. 
# We have 30% of people where the delta with previous rental last 6h to 12h. 
# Then from 1h to 3h is the delay we saw previously that next driver can 'accept' the late. 
# From 3h to 6h represent 16% that we can understand that is because the rate doest represent that more late (only 6%)

: 

#### 1.4.2.1.9. Graphique 10 : Checkin_type repartition on the state 'canceled'

In [ ]:
# Looking for the check-in with cancelling

: 

In [ ]:
# Graph 10 - Now, we gonna see from late rental, if check-in type have an impact on late rental
# We gonna used data_F5 and x = 'checkin_type'

fig, ax = plt.subplots(figsize=(10,5))

sns.countplot(data=data_F5, x="checkin_type")                                                    
ax.set_xticklabels(ax.get_xticklabels(), rotation=70);
ax.set_title("Checkin_type repartition on the state 'canceled'");
total = float(len(data_F5))
for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='center')

: 

In [ ]:
# Graph 10 - Quick data analysis :
# From all cancelation the most chekin-in type is 'mobile' with 77%.

: 

### Answer to the question

In [ ]:
### How does it impact the next driver ? ###

######## RECALL GRAPH ANALYSIS ##########
## Graph 2 - Quick data analysis :
# * We can see that all of driver keep their rental even if they are late
# * We can conclude that all next driver are unsatisfied.
# --------------------------------------------------------------------------------------------------
# Graph 3 - Quick data analysis :
# * We can see that most of delay are under 3h late
# * 53% 'Less than an hours'
# * 33 % "1h to 3h"
# --------------------------------------------------------------------------------------------------
# Graph 4 - Quick data analysis :
# * We can see that mostly when the delta is from 6h to 12h, the car rental are late at 30%
# * Then, it is the time delta of 1h to 3h with 26%
# * Both, represent 56% of late return.
# * When the time delta of 'Less than an hours', we have only 7% of late return.
# * and 13% are late when they don't have time delta
# * We can conclude that if the time delta is short, there are less late return.
# --------------------------------------------------------------------------------------------------
# Graph 5 - Quick data analysis :
# * Mostly people are late bellow 3h. It represent 80% of late !
# --------------------------------------------------------------------------------------------------
# Graph 6 - Quick data analysis :
# Here we can analyze that mostly of the 'canceled' state, had a gap of 6h to 12h at 35% and 17% with a gap from 3h to 6h. 
# We can notice that this is not only because le previous rental are late. But they just canceled for unknown reason.
# But from no time delta to 3h late represent 42% of cancelation
# To avoid late cancelation, we should put a threshold on it. 
# The threshold of cancelation should be 24h before the time departure
# --------------------------------------------------------------------------------------------------
# Graph 7 - Quick data analysis :
# Just to check the checkin_type distribution and as they are almost 50/50, this is not a feature that affect delay.
# --------------------------------------------------------------------------------------------------
# Graph 8 - Quick data analysis :
# Most of people who keep their renting, can still rent the car until 3h late. it represent 87% under all delay_types
# --------------------------------------------------------------------------------------------------
# Graph 9 - Quick data analysis :
# Initially, people keep their renting because the time_delta is longer than other. 
# We have 30% of people where the delta with previous rental last 6h to 12h. 
# Then from 1h to 3h is the delay we saw previously that next driver can 'accept' the late. 
# From 3h to 6h represent 16% that we can understand that is because the rate doest represent that more late (only 6%)
# --------------------------------------------------------------------------------------------------
# Graph 10 - Quick data analysis :
# From all cancelation the most chekin-in type is 'mobile' with 77%.
# -----
######## ANSWER TO THE QUESTION ##########
# All previous_rental have the state 'ended', that mean that all previous rental keep their rental even if they are late
# They are unsatisfied by late arrival. 


: 

## 1.4.3. Question 3 : How many problematic cases will it solve depending on the chosen threshold and scope ?

In [ ]:
# Setting the threshold and the scope 

: 

In [ ]:
# From the analysis above :
# * we can see that a threshold of 3h between rental car will solve 86 % of late rental 
# * we saw that cancelation does't came from only for late rental, but as well for unknown reason. 

# ==> **Threshold** : 
#  * Minimum time between two rental will be set up at 3h
#  * Minimum canceling would be 24h before the time rental

#  ==> **Scope** :
#  * Even it is almost 50/50 with previous rental, the type of checking is up to 77% for mobile. 
#    We should have an action on it. But need to know more about this feature.

: 

## 1.4.4. Question 4 : How many rentals would be affected by the feature depending on the threshold and scope we choose ?

In [ ]:
# So, now, we gonna measure the impact of those feature (Threshold and Scope)

: 

In [ ]:
# ==> **Threshold** : 
#  * Minimum time between two rental will be set up at 3h : 
#    => It will solve : 86% late_rental (Graph 8)
#  * Minimum canceling would be 24h before the time rental
#    => It will solve : 100% of canceling (Graph 6)

#  ==> **Scope** :
#  * Even it is almost 50/50 with previous rental, the type of checking is up to 77% for mobile. 
#    We should have an action on it. But need to know more about this feature.

: 

In [ ]:
# As we can see, 

: 

## 1.4.5. Question 5 : Which share of our owner's revenue would potentially be affected by the feature ?

In [ ]:
# The marketing
# App developpement with 'mobile' and 'connect' 
# Legal
# Lawyer

: 